# Text Classification with From-Scratch Transformer

This notebook implements:
1. A complete transformer-style classifier from scratch (Part b)
2. An efficient attention variant (Linear Attention) with comparisons (Part c)

In [24]:
!pip install numpy pandas matplotlib torch scikit-learn gensim nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 839.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [28]:
!pip install --upgrade numpy gensim

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import time
import math

nltk.download('punkt')
nltk.download('stopwords')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)


## Part B: From-Scratch Transformer Classifier

### 1. Data Loading and Preprocessing

In [ ]:
df = pd.read_csv('train.csv')  # Update with your path
print(f"Dataset shape: {df.shape}")
print(df.head())

# Preprocessing functions
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Apply preprocessing
df['processed_text'] = df['Text'].apply(preprocess_text)

# Encode labels
label_map = {label: idx for idx, label in enumerate(df['Category'].unique())}
df['label'] = df['Category'].map(label_map)

# Split data
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")
print(f"Number of classes: {len(label_map)}")


In [ ]:
sentences = df['processed_text'].tolist()
w2v_model = Word2Vec(sentences=sentences, vector_size=128, window=5, min_count=1, workers=4)
w2v_model.train(sentences, total_examples=len(sentences), epochs=10)

# Create embedding matrix
vocab_size = len(w2v_model.wv)
embedding_dim = w2v_model.vector_size
embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))  # +1 for padding token

for i in range(len(w2v_model.wv)):
    embedding_matrix[i] = w2v_model.wv[i]

# Padding token is at index vocab_size
embedding_matrix[vocab_size] = np.zeros(embedding_dim)

# Token to index mapping
word_to_idx = {word: idx for idx, word in enumerate(w2v_model.wv.index_to_key)}
word_to_idx['<PAD>'] = vocab_size  # Add padding token

In [ ]:
class TextDataset(Dataset):
    def __init__(self, dataframe, word_to_idx, max_len=128):
        self.data = dataframe
        self.word_to_idx = word_to_idx
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['processed_text']
        label = self.data.iloc[idx]['label']

        # Convert tokens to indices
        indices = [self.word_to_idx.get(token, self.word_to_idx['<PAD>']) for token in text]

        # Pad or truncate
        if len(indices) < self.max_len:
            indices = indices + [self.word_to_idx['<PAD>']] * (self.max_len - len(indices))
        else:
            indices = indices[:self.max_len]

        return torch.LongTensor(indices), torch.tensor(label, dtype=torch.long)

# Create datasets and dataloaders
max_len = 128
batch_size = 32

train_dataset = TextDataset(train_df, word_to_idx, max_len)
val_dataset = TextDataset(val_df, word_to_idx, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


### 2. Transformer Components Implementation

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = F.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output


In [ ]:
# Feed Forward Network
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x


In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=2048):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, x, x, mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)

        ffn_output = self.ffn(x)
        x = x + self.dropout2(ffn_output)
        x = self.norm2(x)

        return x


In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, num_layers, num_heads, max_len):
        super(TransformerClassifier, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim, padding_idx=vocab_size)

        # Initialize with Word2Vec embeddings
        self.embedding.weight.data[:-1] = torch.from_numpy(embedding_matrix[:-1])
        self.embedding.weight.requires_grad = True  # Fine-tune embeddings

        # Positional encoding
        self.positional_encoding = PositionalEncoding(embedding_dim, max_len)

        # Transformer layers
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(embedding_dim, num_heads)
            for _ in range(num_layers)
        ])

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(embedding_dim // 2, num_classes)
        )

    def forward(self, x):
        # Embedding and positional encoding
        x = self.embedding(x)
        x = self.positional_encoding(x)

        # Transformer layers
        for layer in self.encoder_layers:
            x = layer(x)

        # Use the [CLS] token equivalent (mean pooling in our case)
        x = x.mean(dim=1)

        # Classification
        x = self.classifier(x)
        return x

### 3. Training and Evaluation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(label_map)
num_layers = 3
num_heads = 4

model = TransformerClassifier(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    num_classes=num_classes,
    num_layers=num_layers,
    num_heads=num_heads,
    max_len=max_len
).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

# %%
# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in dataloader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    return avg_loss, accuracy, precision, recall, f1

# Validation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    return avg_loss, accuracy, precision, recall, f1

# %%
# Training loop
num_epochs = 15
train_losses = []
val_losses = []
train_metrics = []
val_metrics = []

best_f1 = 0
best_model = None

for epoch in range(num_epochs):
    start_time = time.time()

    # Train
    train_loss, train_acc, train_prec, train_rec, train_f1 = train_epoch(
        model, train_loader, criterion, optimizer, device
    )

    # Evaluate
    val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate(
        model, val_loader, criterion, device
    )

    # Update scheduler
    scheduler.step(val_f1)

    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_metrics.append((train_acc, train_prec, train_rec, train_f1))
    val_metrics.append((val_acc, val_prec, val_rec, val_f1))

    # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_model = model.state_dict()

    # Print progress
    elapsed = time.time() - start_time
    print(f"Epoch {epoch+1}/{num_epochs} - {elapsed:.2f}s")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    print(f"Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f}")
    print("-" * 60)

# Load best model
model.load_state_dict(best_model)

# %%
# Plot training curves
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
train_accs = [m[0] for m in train_metrics]
val_accs = [m[0] for m in val_metrics]
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()

# %%
# Final evaluation
val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, criterion, device)
print(f"Final Validation Metrics:")
print(f"Loss: {val_loss:.4f}")
print(f"Accuracy: {val_acc:.4f}")
print(f"Precision: {val_prec:.4f}")
print(f"Recall: {val_rec:.4f}")
print(f"F1 Score: {val_f1:.4f}")


# Linear Attention Implementation
class LinearAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(LinearAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        # Feature map for linear attention
        self.feature_map = nn.Sequential(
            nn.Linear(self.d_k, self.d_k),
            nn.ReLU()
        )

    def elu_feature_map(self, x):
        return F.elu(x) + 1

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        # Project queries, keys, values
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Apply feature map
        Q = self.elu_feature_map(Q)
        K = self.elu_feature_map(K)

        # Compute KV matrix first (more efficient for longer sequences)
        KV = torch.einsum('bhnd,bhne->bhde', K, V)

        # Compute numerator
        Z = 1 / (torch.einsum('bhnd,bhd->bhn', Q, K.sum(dim=2)) + 1e-6)
        V = torch.einsum('bhn,bhde,bhnd->bhne', Z, KV, Q)

        # Combine heads
        V = V.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(V)

        return output

# %%
# Transformer Encoder Layer with Linear Attention
class LinearTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=2048):
        super(LinearTransformerEncoderLayer, self).__init__()
        self.self_attn = LinearAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, x, x, mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)

        ffn_output = self.ffn(x)
        x = x + self.dropout2(ffn_output)
        x = self.norm2(x)

        return x

# %%
# Transformer Model with Linear Attention
class LinearTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, num_layers, num_heads, max_len):
        super(LinearTransformerClassifier, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim, padding_idx=vocab_size)
        self.embedding.weight.data[:-1] = torch.from_numpy(embedding_matrix[:-1])
        self.embedding.weight.requires_grad = True

        # Positional encoding
        self.positional_encoding = PositionalEncoding(embedding_dim, max_len)

        # Transformer layers with linear attention
        self.encoder_layers = nn.ModuleList([
            LinearTransformerEncoderLayer(embedding_dim, num_heads)
            for _ in range(num_layers)
        ])

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(embedding_dim // 2, num_classes)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)

        for layer in self.encoder_layers:
            x = layer(x)

        x = x.mean(dim=1)
        x = self.classifier(x)
        return x

# %%
# Initialize linear attention model
linear_model = LinearTransformerClassifier(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    num_classes=num_classes,
    num_layers=num_layers,
    num_heads=num_heads,
    max_len=max_len
).to(device)

# Loss and optimizer
linear_criterion = nn.CrossEntropyLoss()
linear_optimizer = optim.AdamW(linear_model.parameters(), lr=1e-4, weight_decay=1e-5)
linear_scheduler = optim.lr_scheduler.ReduceLROnPlateau(linear_optimizer, mode='max', factor=0.5, patience=2)

# %%
# Training loop for linear attention
linear_train_losses = []
linear_val_losses = []
linear_train_metrics = []
linear_val_metrics = []

linear_best_f1 = 0
linear_best_model = None

for epoch in range(num_epochs):
    start_time = time.time()

    # Train
    train_loss, train_acc, train_prec, train_rec, train_f1 = train_epoch(
        linear_model, train_loader, linear_criterion, linear_optimizer, device
    )

    # Evaluate
    val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate(
        linear_model, val_loader, linear_criterion, device
    )

    # Update scheduler
    linear_scheduler.step(val_f1)

    # Store metrics
    linear_train_losses.append(train_loss)
    linear_val_losses.append(val_loss)
    linear_train_metrics.append((train_acc, train_prec, train_rec, train_f1))
    linear_val_metrics.append((val_acc, val_prec, val_rec, val_f1))

    # Save best model
    if val_f1 > linear_best_f1:
        linear_best_f1 = val_f1
        linear_best_model = linear_model.state_dict()

    # Print progress
    elapsed = time.time() - start_time
    print(f"Epoch {epoch+1}/{num_epochs} - {elapsed:.2f}s")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    print(f"Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f}")
    print("-" * 60)

# Load best model
linear_model.load_state_dict(linear_best_model)

# %%
# Compare standard and linear attention
def compare_models(standard_model, linear_model, dataloader, device):
    # Standard attention
    start_time = time.time()
    with torch.no_grad():
        for batch in dataloader:
            inputs, _ = batch
            inputs = inputs.to(device)
            _ = standard_model(inputs)
    standard_time = time.time() - start_time

    # Linear attention
    start_time = time.time()
    with torch.no_grad():
        for batch in dataloader:
            inputs, _ = batch
            inputs = inputs.to(device)
            _ = linear_model(inputs)
    linear_time = time.time() - start_time

    # Memory usage
    standard_mem = torch.cuda.max_memory_allocated(device) if torch.cuda.is_available() else 0
    torch.cuda.reset_peak_memory_stats(device)

    with torch.no_grad():
        for batch in dataloader:
            inputs, _ = batch
            inputs = inputs.to(device)
            _ = linear_model(inputs)
    linear_mem = torch.cuda.max_memory_allocated(device) if torch.cuda.is_available() else 0

    return {
        'standard_time': standard_time,
        'linear_time': linear_time,
        'standard_mem': standard_mem,
        'linear_mem': linear_mem
    }

# Performance comparison
comparison = compare_models(model, linear_model, val_loader, device)
print("\nPerformance Comparison:")
print(f"Standard Attention Time: {comparison['standard_time']:.4f}s")
print(f"Linear Attention Time: {comparison['linear_time']:.4f}s")
print(f"Speedup: {comparison['standard_time'] / comparison['linear_time']:.2f}x")
print(f"Standard Attention Memory: {comparison['standard_mem'] / 1024**2:.2f}MB")
print(f"Linear Attention Memory: {comparison['linear_mem'] / 1024**2:.2f}MB")

# %%
# Final evaluation of linear attention model
val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate(linear_model, val_loader, linear_criterion, device)
print(f"\nLinear Attention Validation Metrics:")
print(f"Loss: {val_loss:.4f}")
print(f"Accuracy: {val_acc:.4f}")
print(f"Precision: {val_prec:.4f}")
print(f"Recall: {val_rec:.4f}")
print(f"F1 Score: {val_f1:.4f}")


"""
## Analysis of Linear Attention

### Mathematical Basis for Efficiency Gains

Standard attention computes:
\[ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V \]

This has O(N^2) complexity due to the QK^T matrix multiplication.

Linear attention reformulates this as:
\[ \text{Attention}(Q, K, V) = \phi(Q)(\phi(K)^T V \]
where φ is a feature map (we used ELU(x)+1).

By associativity, we can compute (φ(K)^T V) first, reducing complexity to O(N).

### Theoretical Advantages and Limitations

Advantages:
1. Linear complexity with sequence length (O(N) vs O(N^2))
2. Better memory efficiency
3. Can handle much longer sequences

Limitations:
1. Approximation may lose some expressive power
2. Feature map choice is crucial for performance
3. May require more layers to achieve similar performance

### Implementation Complexity

The implementation adds:
1. Feature map application (simple feed-forward)
2. Changed computation order (KV product first)
3. Normalization term (Z)

Overall complexity increase is modest (~20% more code) for significant speed gains.
"""